# Experiment with yolo models to obtain better football detection

## Imports

In [1]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}"

In [2]:
from pathlib import Path
import cv2
import hydra
from hydra import compose, initialize
import supervision as sv

In [3]:
import torch

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

GPU Found!!


## Paths setup

In [4]:
from omegaconf import DictConfig

with initialize(config_path="../configs", job_name="EDA", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg.paths)

{'root_dir': '${oc.env:PROJECT_ROOT}', 'root_data_dir': '${oc.env:DATA_ROOT}', 'results_dir': '${paths.root_dir}/results', 'log_dir': '${paths.root_dir}/logs/', 'output_dir': '${hydra:runtime.output_dir}', 'work_dir': '${hydra:runtime.cwd}', 'pretrained_model_dir': '${paths.root_dir}/pretrained_models'}


In [5]:
cfg.paths.pretrained_model_dir

'/workspaces/football-players-tracking-yolo/pretrained_models'

In [6]:
# auto reload libs
%load_ext autoreload
%autoreload 2

## Download Datasets

In [7]:
ROOT_DIR = Path(cfg.paths.root_dir)
ROOT_DATA_DIR = Path(cfg.paths.root_data_dir)
DATA_DIR = ROOT_DATA_DIR / cfg.datasets.datasets_dir
DATASET  = cfg.datasets.roboflow
DATA_DIR.mkdir(parents=True, exist_ok=True)
DATASET_DIR = DATA_DIR/ cfg.datasets.dataset_name
CLASS_NAMES = cfg.datasets.names
PRETRAINED_MODEEL_DIR = Path(cfg.paths.pretrained_model_dir)

In [8]:
if len(list(DATA_DIR.iterdir())) == 0:
    from roboflow import Roboflow

    rf = Roboflow()
    project = rf.workspace(DATASET.workspace).project(DATASET.project)
    version = project.version(DATASET.version)
    dataset = version.download(model_format=DATASET.model)

## Paths Setup

In [9]:
TRAIN_IMAGES_DIR = DATASET_DIR / "train" / "images"
TRAIN_LABELS_DIR = DATASET_DIR / "train"  / "labels"

VALID_IMAGES_DIR = DATASET_DIR / "valid" / "images"
VALID_LABELS_DIR = DATASET_DIR / "valid"  / "labels"

TEST_IMAGES_DIR = DATASET_DIR / "test" / "images"
TEST_LABELS_DIR = DATASET_DIR / "test"  / "labels"

## YOLO model import

In [10]:
MODEL_NAME = f'{PRETRAINED_MODEEL_DIR}/{cfg.model.model_name.lower()}.pt'
MODEL_NAME

'/workspaces/football-players-tracking-yolo/pretrained_models/yolov8l.pt'

In [11]:
from ultralytics import YOLO
model = YOLO(MODEL_NAME)

In [12]:
results = model.train(data=cfg.datasets.dataset_yaml,
            **cfg.trainer.params,
            **cfg.datasets.augmentations,
)
            

Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3080, 9898MiB)
engine/trainer: task=detect, mode=train, model=/workspaces/football-players-tracking-yolo/pretrained_models/yolov8l.pt, data=/workspaces/football-players-tracking-yolo/data/football-players-detection-14/data.yaml, epochs=200, time=None, patience=20, batch=-1, imgsz=640, save=True, save_period=-1, cache=disk, device=0, workers=16, project=/workspaces/football-players-tracking-yolo/results, name=augumented-data-yolov8l, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=F

E0000 00:00:1745072133.264787   30077 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745072133.268754   30077 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745072133.279663   30077 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745072133.279673   30077 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745072133.279674   30077 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745072133.279676   30077 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

train: Scanning /workspaces/football-players-tracking-yolo/data/football-players-detection-14/train/labels.cache... 298 images, 0 backgrounds, 0 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]
train: Caching images (0.3GB Disk): 100%|██████████| 298/298 [00:00<00:00, 2167.24it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3080) 9.67G total, 0.39G reserved, 0.37G allocated, 8.90G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    43632924       165.4         2.638         32.99         210.8        (1, 3, 640, 640)                    list
    43632924       330.8         3.366         21.99         104.7        (2, 3, 640, 640)                    list
    43632924       661.7         4.853          31.5         107.1        (4, 3, 640, 640)                    list
    43632924        1323         7.766         60.03         140.1        (8, 3, 640, 640)                    list
CUDA out of memory. Tr

...

    179/200      4.09G     0.7275     0.3443     0.7948         48        640: 100%|██████████| 75/75 [00:08<00:00,  9.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00, 14.10it/s]

                   all         49       1174      0.901      0.757      0.817      0.549
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 159, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



179 epochs completed in 0.475 hours.
Optimizer stripped from /workspaces/football-players-tracking-yolo/results/augumented-data-yolov8l/weights/last.pt, 87.7MB
Optimizer stripped from /workspaces/football-players-tracking-yolo/results/augumented-data-yolov8l/weights/best.pt, 87.7MB

Validating /workspaces/football-players-tracking-yolo/results/augumented-data-yolov8l/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3080, 9898MiB)
Model summary (fused): 112 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.90it/s]


                   all         49       1174      0.911      0.759      0.834      0.556
                  ball         45         45      0.889      0.355       0.48      0.217
            goalkeeper         38         39      0.846      0.848      0.911       0.61
                player         49        973      0.967      0.935      0.986      0.764
               referee         49        117      0.941      0.897      0.959      0.634
Speed: 0.1ms preprocess, 11.9ms inference, 0.0ms loss, 8.0ms postprocess per image
Results saved to /workspaces/football-players-tracking-yolo/results/augumented-data-yolov8l


In [18]:
results

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7072dc0d8750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [13]:
# YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs
#                  Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.12it/s]
#                    all         49       1174      0.814       0.73      0.791      0.516
#                   ball         45         45      0.701       0.26      0.341       0.14
#             goalkeeper         38         39      0.791      0.846      0.917      0.615
#                 player         49        973      0.932       0.95      0.981      0.731
#                referee         49        117      0.831      0.863      0.926      0.579

In [14]:
# YOLOv12l summary (fused): 283 layers, 26,342,156 parameters, 0 gradients, 88.6 GFLOPs
#                  Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.92it/s]
#                    all         49       1174       0.86       0.75       0.81      0.537
#                   ball         45         45      0.783      0.333      0.415      0.154
#             goalkeeper         38         39      0.837      0.872      0.908      0.622
#                 player         49        973      0.964      0.928      0.985      0.755
#                referee         49        117      0.856      0.865      0.932      0.616

In [15]:
# model = YOLO('/workspaces/football-players-tracking-yolo/results/augumented-data-yolo12l/weights/best.pt')
# results = model.train(data=cfg.datasets.dataset_yaml,
#             **cfg.trainer.params,
#             **cfg.datasets.augmentations,
# )